<a href="https://colab.research.google.com/github/limshaocong/ReqBERT/blob/main/t3_finetuning_seqlabel_er.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preliminaries**

The following libraries are required for this notebook. If run locally, it can be commented out.


* `transformers` - an excellent library by Hugging Face that provides a high-level abstraction for fine-tuning of transformer-based models, and a model hub of pre-trained models
* `datasets` - a dataset library by Hugging Face. All datasets used for this thesis are uploaded to this dataset hub. 
* `torch` - PyTorch. When running the `allenai/scibert_scivocab_cased` pre-trained model with this notebook on GCP Vertex AI, `torch` needs to be installed.
* `seqeval` - a library for evaluation of sequence labelling tasks

In [ ]:
! pip install --user datasets transformers torch seqeval

While Google Colab provides free GPU access, the specific GPU (K80/ T4/ P100) available is a function of the subscription level (Free Tier/ Colab Pro/ Colab Pro+) and usage trends. Alternatively, Google Cloud Platform provides US$300 in free credits that can be used for paid GPU access.

In [ ]:
! nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-aa64646c-4a4d-0d88-abeb-0833b167a105)


# **Import & Pre-process Data**

These dictionaries will minimize the amount of adjustment within the remainder of the script.

In [ ]:
model_type_dict = {
    'bert-base-cased' : 'bert-base-cased',
    'roberta-base' : 'roberta-base',
    'allenai/scibert_scivocab_cased' : 'allenai/scibert_scivocab_cased',
    'limsc/reqbert-tapt-epoch29' : 'bert-base-cased',
    'limsc/reqroberta-tapt-epoch43' : 'roberta-base',
    'limsc/reqscibert-tapt-epoch20' : 'allenai/scibert_scivocab_cased',
}

model_name_dict = {
    'bert-base-cased' : 'bert',
    'roberta-base' : 'roberta',
    'allenai/scibert_scivocab_cased' : 'scibert',
    'limsc/reqbert-tapt-epoch29' : 'reqbert-e29',
    'limsc/reqroberta-tapt-epoch43' : 'reqroberta-e43',
    'limsc/reqscibert-tapt-epoch20' : 'reqscibert-e20'
}

task_name_dict = {
    'limsc/fr-nfr-classification' : 'frnfr',
    'limsc/subclass-classification' : 'subclass',
    'limsc/concept-recognition-not-iob' : 'cr',
    'limsc/requirements-entity-recognition' : 'er'
}

The dataset for the [entity recognition dataset](https://huggingface.co/datasets/limsc/requirements-entity-recognition) is uploaded onto the HuggingFace hub.

In [ ]:
from datasets import load_dataset

ds_name = 'limsc/requirements-entity-recognition'
ds = load_dataset(ds_name)
ds

Downloading:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

Using custom data configuration limsc--requirements-entity-recognition-772abc9e4a496786


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/limsc___parquet/limsc--requirements-entity-recognition-772abc9e4a496786/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 646
    })
    val: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 138
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 139
    })
})

In [ ]:
label_list = ds['train'].features['ner_tags'].feature.names
label_list

['O',
 'B-ACT',
 'I-ACT',
 'B-ATTR',
 'I-ATTR',
 'B-RELOP',
 'I-RELOP',
 'B-QUANT',
 'I-QUANT',
 'B-ENT',
 'I-ENT']

In [ ]:
model_checkpoint = 'bert-base-cased'

In [ ]:
from transformers import AutoTokenizer

prefix_space = True if model_type_dict[model_checkpoint] == 'roberta-base' else False

tokenizer = AutoTokenizer.from_pretrained(
    model_type_dict[model_checkpoint],
    use_fast = True,
    add_prefix_space = prefix_space
)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens = False):

    tokenized_inputs = tokenizer(
        examples['tokens'],
        truncation = True,
        is_split_into_words = True
    )
    labels = []
    
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        
        for word_idx in word_ids:           
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.

            if word_idx is None:
                label_ids.append(-100)
            
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    
    return tokenized_inputs

In [ ]:
tokenized_ds = ds.map(tokenize_and_align_labels, batched = True)

Parameter 'function'=<function tokenize_and_align_labels at 0x7faf1efdf560> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

# **Model Fine-tuning (Single Loop)**

In [ ]:
from transformers import DataCollatorForTokenClassification

batch_size = 16

data_collator = DataCollatorForTokenClassification(
    tokenizer = tokenizer,
    padding = True,
    return_tensors = 'tf'
)

def batching(tokenized_ds, batch_size):

    batched_train_ds = tokenized_ds['train'].to_tf_dataset(
        columns = ['attention_mask', 'input_ids', 'labels'],
        shuffle = True,
        batch_size = batch_size,
        collate_fn = data_collator,
    )

    batched_val_ds = tokenized_ds['val'].to_tf_dataset(
        columns = ['attention_mask', 'input_ids', 'labels'],
        shuffle = False,
        batch_size = batch_size,
        collate_fn = data_collator,
    )

    batched_test_ds = tokenized_ds['test'].to_tf_dataset(
        columns = ['attention_mask', 'input_ids', 'labels'],
        shuffle = False,
        batch_size = batch_size,
        collate_fn = data_collator,
    )
    
    return batched_train_ds, batched_val_ds, batched_test_ds

batched_train_ds, batched_val_ds, batched_test_ds = batching(tokenized_ds, batch_size)

/root/.local/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForTokenClassification, create_optimizer

seed = 6789767
tf.random.set_seed(seed)

num_epochs = 5
initial_lr = 2e-5

def create_model(num_epochs, initial_lr):

    from_pt = True if model_checkpoint == 'allenai/scibert_scivocab_cased' else False

    model = TFAutoModelForTokenClassification.from_pretrained(
        model_checkpoint,
        num_labels = len(label_list),
        from_pt = from_pt
    )

    batches_per_epoch = len(tokenized_ds['train']) // batch_size
    total_train_steps = int(batches_per_epoch * num_epochs)

    optimizer, schedule = create_optimizer(
        init_lr = initial_lr,
        num_warmup_steps = total_train_steps // 20,
        num_train_steps = total_train_steps,
        weight_decay_rate = 0.01
    )

    model.compile(optimizer = optimizer)

    return model

model = create_model(num_epochs, initial_lr)

In [ ]:
import numpy as np
from datasets import load_metric
from transformers.keras_callbacks import KerasMetricCallback

metric = load_metric('seqeval')

def compute_metrics(p):

    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions = true_predictions, references = true_labels)

    return {
        'precision': results['overall_precision'],
        'recall': results['overall_recall'],
        'f1': results['overall_f1'],
        'accuracy': results['overall_accuracy'],
    }

metric_cb = KerasMetricCallback(
    metric_fn = compute_metrics,
    eval_dataset = batched_val_ds
)

In [ ]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, CSVLogger

class update_logger(Callback):

    def __init__(self):    
        super(update_logger, self).__init__()

    def on_epoch_end(self, epoch, logs = {}):
        logs['seed'] = seed
        logs['batch_size'] = batch_size
        logs['learning_rate'] = initial_lr

update_logger_cb = update_logger()

csvlogger_file = f'{task_name_dict[ds_name]}-{model_name_dict[model_checkpoint]}.csv'
csvlogger_cb = CSVLogger(csvlogger_file, append = True)

callbacks = [metric_cb, update_logger_cb, csvlogger_cb]

In [ ]:
model.fit(
    batched_train_ds,
    validation_data = batched_val_ds,
    epochs = num_epochs,
    callbacks = callbacks
)

# **Hyperparameter tuning**

In [ ]:
batch_sizes = [8]
initial_lrs = [5e-5, 3e-5, 2e-5]
seeds = [21916, 25412, 56281, 61712, 30488,
         28215, 78867, 87843, 67918, 93327,
         95420, 11905, 86349, 12082, 81996]

num_epochs = 5

for batch_size in batch_sizes:

    batched_train_ds, batched_val_ds, batched_test_ds = batching(tokenized_ds, batch_size)
    
    for initial_lr in initial_lrs:
    
        for seed in seeds:
    
            tf.random.set_seed(seed)
            model = create_model(num_epochs, initial_lr)

            csvlogger_file = f'{task_name_dict[ds_name]}-{model_name_dict[model_checkpoint]}-bs{batch_size}.csv'
            csvlogger_cb = CSVLogger(csvlogger_file, append = True)

            callbacks = [metric_cb, update_logger_cb, csvlogger_cb]

            model.fit(
                batched_train_ds,
                validation_data = batched_val_ds,
                epochs = num_epochs,
                callbacks = callbacks
            )

# **Evaluate on test set**

In [ ]:
mod_hyperparam_dict = {
    'bert-base-cased' : {'batch_size' : 8, 'initial_lr' : 3e-5, 'num_epochs' : 2},
    'roberta-base' : {'batch_size' : 8, 'initial_lr' : 5e-5, 'num_epochs' : 2},
    'allenai/scibert_scivocab_cased' : {'batch_size' : 8, 'initial_lr' : 3e-5, 'num_epochs' : 2},
    'limsc/reqbert-tapt-epoch29' : {'batch_size' : 8, 'initial_lr' : 3e-5, 'num_epochs' : 2},
    'limsc/reqroberta-tapt-epoch43' : {'batch_size' : 8, 'initial_lr' : 5e-5, 'num_epochs' : 2},
    'limsc/reqscibert-tapt-epoch20' : {'batch_size' : 8, 'initial_lr' : 3e-5, 'num_epochs' : 2}
}

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

mods = list(mod_hyperparam_dict.keys())
results_list = []
seeds = [21916, 25412, 56281, 61712, 30488,
         28215, 78867, 87843, 67918, 93327,
         95420, 11905, 86349, 12082, 81996]

for mod in mods:

    model_checkpoint = mod

    batch_size = mod_hyperparam_dict[mod]['batch_size']
    initial_lr = mod_hyperparam_dict[mod]['initial_lr']
    num_epochs = mod_hyperparam_dict[mod]['num_epochs']

    prefix_space = True if model_type_dict[model_checkpoint] == 'roberta-base' else False

    tokenizer = AutoTokenizer.from_pretrained(
        model_type_dict[model_checkpoint],
        use_fast = True,
        add_prefix_space = prefix_space
    )

    tokenized_ds = ds.map(tokenize_and_align_labels, batched = True)
    
    data_collator = DataCollatorForTokenClassification(
        tokenizer = tokenizer,
        padding = True,
        return_tensors = 'tf'
    )

    batched_train_ds, batched_val_ds, batched_test_ds = batching(tokenized_ds, batch_size)

    for seed in seeds:

        tf.random.set_seed(seed)
        model = create_model(num_epochs, initial_lr)

        model.fit(
            batched_train_ds,
            epochs = num_epochs
        )

        all_preds = []
        all_trues = []

        for batch in batched_test_ds:

            y_preds_logits = model.predict(batch)['logits']
            y_preds = np.argmax(y_preds_logits, axis = 2)
            y_trues = batch['labels']
            p = y_preds, y_trues

            all_pred = [
                [label_list[p] for (p, l) in zip(y_pred, y_true) if l != -100]
                for y_pred, y_true in zip(y_preds, y_trues)]
            all_true = [
                [label_list[l] for (p, l) in zip(y_pred, y_true) if l != -100]
                for y_pred, y_true in zip(y_preds, y_trues)]
           
            rep = metric.compute(
                predictions = all_pred,
                references = all_true
            )

            result_dict = {
                'model': mod,
                'seed' : seed,
                'ACT_p' : format(rep['ACT']['precision'], '.4f'),
                'ACT_r' : format(rep['ACT']['recall'], '.4f'),
                'ACT_f1' : format(rep['ACT']['f1'], '.4f'),
                'ATTR_p' : format(rep['ATTR']['precision'], '.4f'),
                'ATTR_r' : format(rep['ATTR']['recall'], '.4f'),
                'ATTR_f1' : format(rep['ATTR']['f1'], '.4f'),
                'ENT_p' : format(rep['ENT']['precision'], '.4f'),
                'ENT_r' : format(rep['ENT']['recall'], '.4f'),
                'ENT_f1' : format(rep['ENT']['f1'], '.4f'),
                'QUANT_p' : format(rep['QUANT']['precision'], '.4f'),
                'QUANT_r' : format(rep['QUANT']['recall'], '.4f'),
                'QUANT_f1' : format(rep['QUANT']['f1'], '.4f'),
                'RELOP_p' : format(rep['RELOP']['precision'], '.4f'),
                'RELOP_r' : format(rep['RELOP']['recall'], '.4f'),
                'RELOP_f1' : format(rep['RELOP']['f1'], '.4f'),
                'overall_accuracy' : format(rep['overall_accuracy'], '.4f'),
                'overall_f1' : format(rep['overall_f1'], '.4f'),
                'overall_p' : format(rep['overall_precision'], '.4f'),
                'overall_r' : format(rep['overall_recall'], '.4f')
            }

            results_list.append(result_dict)

In [ ]:
import pandas as pd

test_df = pd.DataFrame(results_list)
test_filename = f'{task_name_dict[ds_name]}-test.csv'
test_df.to_csv(test_filename)

# **Prediction Pipeline**

In [ ]:
from transformers import TokenClassificationPipeline

pipe = TokenClassificationPipeline(
    model = model,
    tokenizer = tokenizer,
    # aggregation_strategy = 'simple'
)

In [ ]:
text = 'The Micro-VCM datasheet shall contain the product type.'
text = 'Storage conditions shall prevent the degredation of the structure.'
text = 'A record of the process data shall be part of the process procedure'
text = 'In case of doubt, the internal NRB shall classify nonconformances as major.'
text = 'The Reserved shall be an 8-bit field that is set to 0x00.'

pipe(text)